In [378]:
# TO generate sql script for inserting mock data into db2

from faker import Faker
import sqlite3
import random

In [379]:
fake = Faker()

In [380]:
num_records = 100
insert_statements = []

In [ ]:
# User
user_email = {}
for _ in range(num_records):
    name = fake.name()
    
    email = fake.email()
    while email in user_email:
        email = fake.email()
    user_email[email] = 1
    
    balance = round(random.uniform(0,100000), 2)
    password = fake.password()
    insert_statements.append(f"INSERT INTO User VALUES ('{email}', {balance}, '{name}', '{password}');")

In [382]:
# Bank
for i in range(num_records):
    insert_statements.append(f"INSERT INTO Bank VALUES ({i+1});")

In [383]:
# BankAccount
bank_account_with_instit = []
for _ in range(num_records):
    institutionNumber = random.randint(1,100)                         # everything from 1 - 100 is generated
    accountNumber = random.randint(100000,100000000)
    bank_account_with_instit.append((institutionNumber, accountNumber))
    name = fake.name()
    balance =  round(random.uniform(0,100000), 2)
    insert_statements.append(f"INSERT INTO BankAccount VALUES ({institutionNumber}, {accountNumber}, '{name}', {balance});")

In [384]:
# BankTransactions
used_transactionID = {}
status_code = ["Complete", "Pending", "Failed", "Not Submitted", "Archived"]
opType = ["Cheque", "Saving" ]
for _ in range(num_records):
    transactionID = random.randint(100000,100000000)
    while transactionID in used_transactionID:
        transactionID = random.randint(100000,100000000)
    used_transactionID[transactionID] = 1

    date = fake.date()
    currency = fake.currency_code()
    amount = round(random.uniform(0,100000), 2)
    status = random.choice(status_code)
    opCode = random.choice(opType)
    institute, bankAccountNumber = random.choice(bank_account_with_instit)
    user = random.choice(list(user_email.keys()))
       
    insert_statements.append(f"INSERT INTO BankTransactions VALUES ({transactionID}, '{date}', '{currency}', {amount}, '{status}', '{opCode}', {institute}, {bankAccountNumber}, '{user}');")

In [385]:
# Company
company_IP = {}
company_name = {}
company_sectors = ["Technology", "Healthcare", "Finance", "Education", "Retail", "Manufacturing", "Real Estate", "Marketing", "Transportation"]
for _ in range(num_records):
    CUSIP = fake.lexify(text='?????????').upper()
    while CUSIP in company_IP:
        CUSIP = fake.lexify(text='?????????').upper()
    company_IP[CUSIP] = 1
    
    name = fake.company()
    while name in company_name:
        name = fake.company()
    company_name[name] = 1
    
    location = fake.address()
    sector = random.choice(company_sectors)
       
    insert_statements.append(f"INSERT INTO Company VALUES ('{CUSIP}', '{name}', '{location}', '{sector}');")

In [386]:
# Stock
used_ticker = {} 
for _ in range(num_records):
    ticker = fake.lexify(text='???').upper()
    while ticker in used_ticker:
        ticker = fake.lexify(text='???').upper()
    used_ticker[ticker] = 1
    marketCap = round(random.uniform(1000000,100000000), 2)
    Company = random.choice(list(company_IP.keys()))
    insert_statements.append(f"INSERT INTO Stock VALUES ('{ticker}', {marketCap}, '{Company}');")

In [387]:
# Shares
used_shareID_with_ticker = {}
stock_exchanges = [
    "NYSE",  # New York Stock Exchange
    "NASDAQ",  # NASDAQ Stock Exchange
    "LSE",  # London Stock Exchange
    "TSE",  # Tokyo Stock Exchange
    "SSE",  # Shanghai Stock Exchange
    "HKEX",  # Hong Kong Stock Exchange
    "Euronext",  # Euronext (Paris, Amsterdam, etc.)
    "ASX",  # Australian Securities Exchange
    "BSE",  # Bombay Stock Exchange
    "TSX",  # Toronto Stock Exchange
]
for _ in range(num_records):
    ticker = random.choice(list(used_ticker.keys()))
    shareID = random.randint(100000,100000000)
    while (shareID, ticker) in used_shareID_with_ticker:
        shareID = random.randint(100000,100000000)
        ticker = random.choice(list(used_ticker.keys()))
    used_shareID_with_ticker[(shareID, ticker)] = 1

    currentP = round(random.uniform(0,1000), 2)
    openingP = round(random.uniform(0,1000), 2)
    stockExchange = random.choice(stock_exchanges)
    insert_statements.append(f"INSERT INTO Shares VALUES ('{ticker}', {shareID}, {currentP}, {openingP}, '{stockExchange}');")

In [388]:
#InvestmentTransactions
for _ in range(num_records):
    transactionID = random.randint(100000,100000000) 
    while transactionID in used_transactionID:                           # same dict as bank transaction 
        transactionID = random.randint(100000,100000000)
    used_transactionID[transactionID] = 1

    date = fake.date()
    currency = fake.currency_code()
    amount = round(random.uniform(0,100000), 2)
    status = random.choice(status_code)
    opCode = random.choice(opType)
    brokerageFee = round(random.uniform(0,10), 2) 

    shareID, ticker = random.choice(list(used_shareID_with_ticker.keys()))
    user = random.choice(list(user_email.keys()))
    
    insert_statements.append(f"INSERT INTO InvestmentTransactions VALUES ({transactionID},'{date}','{currency}',{amount},'{status}','{opCode}',{brokerageFee},'{ticker}',{shareID},'{user}');")

In [389]:
# Watchlist
WID = {}
for _ in range(num_records):
    wid = random.randint(100000,100000000) 
    while wid in WID:                           
        wid = random.randint(100000,100000000)
    WID[wid] = 1

    watchlistname = fake.word()

    notificationMode = random.choice(["On", "Off"])
    
    insert_statements.append(f"INSERT INTO Watchlist VALUES ({wid},'{watchlistname}','{notificationMode}');")

In [390]:
# Account
PID = {}
account_type = ["individual", "TFSA", "RRSP"]
for _ in range(num_records):
    pid = random.randint(100000,100000000) 
    while pid in PID:                           
        pid = random.randint(100000,100000000)
    PID[pid] = 1

    name = fake.word()
    accountT = random.choice(account_type)
    value = round(random.uniform(0,100000), 2)
    watchlist = random.choice(list(WID.keys()))
    user = random.choice(list(user_email.keys()))
   
    insert_statements.append(f"INSERT INTO Account VALUES ({pid}, '{name}', '{accountT}', {value}, {watchlist}, '{user}');")

In [391]:
# PriceHistory
ticker_with_date = {}
for _ in range(num_records):
    ticker = random.choice(list(used_ticker.keys()))
    datetime = fake.date()
    while (ticker, datetime) in ticker_with_date:  
        ticker = random.choice(list(used_ticker.keys()))
        datetime = fake.time()
    ticker_with_date[(ticker, datetime)] = 1

    price = round(random.uniform(0,100000), 2) 
    
    insert_statements.append(f"INSERT INTO PriceHistory VALUES ('{ticker}','{datetime}',{price});")

In [392]:
# WatchlistAndShares
WID_shareID_ticker = {}
for _ in range(num_records):
    shareID, ticker = random.choice(list(used_shareID_with_ticker.keys()))
    wid = random.choice(list(WID.keys()))
    while (shareID, ticker, wid) in WID_shareID_ticker:  
        shareID, ticker = random.choice(list(used_shareID_with_ticker.keys()))
        wid = random.choice(list(WID.keys()))
    WID_shareID_ticker[(shareID, ticker, wid)] = 1

    insert_statements.append(f"INSERT INTO WatchlistAndShares VALUES ({wid}, {shareID}, '{ticker}');")

In [393]:
# AccountAndShares
shareID_ticker = {}
for _ in range(num_records):
    shareID, ticker = random.choice(list(used_shareID_with_ticker.keys()))
    while (shareID, ticker) in shareID_ticker:  
        shareID, ticker = random.choice(list(used_shareID_with_ticker.keys()))
    shareID_ticker[(shareID, ticker)] = 1

    portID = random.choice(list(PID.keys()))
    insert_statements.append(f"INSERT INTO AccountAndShares VALUES ({shareID}, '{ticker}', {portID});")

In [394]:
sql_output = "CONNECT TO COMP421; \n" + "\n".join(insert_statements)
print(sql_output)

CONNECT TO COMP421; 
INSERT INTO User VALUES ('matthew22@example.com', 31341.32, 'Andrew Matthews', '+GFFqY!4J6');
INSERT INTO User VALUES ('ibonilla@example.net', 41859.58, 'Michael Ward', 'uCVmkYqQ+6');
INSERT INTO User VALUES ('nathan48@example.com', 83244.69, 'Raymond Doyle', 'HV99KAnhq)');
INSERT INTO User VALUES ('pcox@example.com', 14491.61, 'Kevin Peters', '(Ak1$Q0at5');
INSERT INTO User VALUES ('dtrujillo@example.com', 34775.36, 'Laurie Sanders', '%nQ9Inwa4u');
INSERT INTO User VALUES ('dennisashley@example.net', 4484.96, 'Kevin Gonzalez', '1TXqXUi4$D');
INSERT INTO User VALUES ('gibsonelizabeth@example.net', 9945.5, 'Stephen Wallace', '#15NJSg_xv');
INSERT INTO User VALUES ('charles56@example.com', 66750.73, 'Nicole Beasley', 'a4Z^gf7Z+c');
INSERT INTO User VALUES ('colemansara@example.com', 13563.73, 'Amber Carroll', '7v_IxLr_$F');
INSERT INTO User VALUES ('amberflores@example.org', 32831.84, 'Sierra Carey', '$0JrdG7rZs');
INSERT INTO User VALUES ('ygarza@example.com', 18927